In [1]:
from orbital_simulation import Rigidbody, Simulation
from astropy.constants import R_earth

from itertools import combinations

import numpy as np
from catppuccin.palette import PALETTE

import matplotlib as mpl

In this Simulation, we will try to perform a simple Hohmann transfer
to send a spacecraft to the orbit of another spacecraft.

The necessary maneuver can be calculated and performed analytically, since the
spacecrafts do not attract eachother, but instead this is a trial and error solution.

In [2]:
sim = Simulation(dt=1)

# Load the Earth from config
earth = Rigidbody.from_name("earth")

# Target Rigidbody in geostationary orbit
target = Rigidbody(
    name="Target",
    # important: since the earth is not stationary,
    # we have to add our positions and velocities to the earth's
    position=earth.get_current_position() + np.array([R_earth.value + 35_786e3, 0, 0]),
    velocity=earth.get_current_velocity() + np.array([0, 3.0746e3, 0]),
    mass=440e3,
    radius=10,
    body_color=PALETTE.latte.colors.green.hex,
    trail_color=PALETTE.frappe.colors.green.hex,
    intrinsic_acceleration=np.zeros(3),
    marker="x",
    fix_marker_size=True,
)

# Spacecraft we try to get to the same orbit as our target
craft = Rigidbody(
    name="Spacecraft",
    # important: since the earth is not stationary,
    # we have to add our positions and velocities to the earth's
    position=earth.get_current_position() + np.array([-(R_earth.value + 400e3), 0, 0]),
    velocity=earth.get_current_velocity() + np.array([0, -7.6e3, 0]),
    mass=440e3,
    radius=10,
    body_color=PALETTE.latte.colors.teal.hex,
    trail_color=PALETTE.frappe.colors.teal.hex,
    intrinsic_acceleration=np.zeros(3),
    marker="x",
    fix_marker_size=True,
)

spacecrafts = [target, craft]

# This is a bit overkill for two spacecrafts, but case of more than
# two bodies, which should not interact with eachother, this combinations
# ansatz is pretty handy.
for craft1, craft2 in combinations(spacecrafts, 2):
    craft1.exclude_body(craft2)

# add the earth to the simulation
sim.add_rigidbody(earth)

# add all spacecrafts to the simulation
for craft in spacecrafts:
    sim.add_rigidbody(craft)

In [ ]:
sim.reset()  # reset in case you rerun this cell

# let the craft orbit a little more than once
sim.run(116.5 * 60)

# accelerate to raise the apoapsis to the target height
craft.accelerate(acceleration=39.66, mode="prograde", relative_to=earth)

# accelerate for 60 seconds
sim.run(60)

# stop accelerating
craft.stop_acceleration()

# wait until the spacecraft reached its apoapsis
sim.run(317 * 60)

# calculate the speed we will have to accelerate to, to match the orbits
delta_v = (
    np.linalg.norm(craft.get_current_velocity() - target.get_current_velocity()) * 0.988
)


acceleration_time = 50
craft.accelerate(
    acceleration=delta_v / acceleration_time, mode="prograde", relative_to=earth
)

# accelerate for the given acceleration time
craft.accelerate(acceleration=1.16, mode="radial_out", relative_to=earth)
sim.run(acceleration_time)
craft.stop_acceleration()

# let the simulation run for 20 hours to see the final orbits.
sim.run(20 * 3600)

Simulating steps:   0%|          | 0/6991 [00:00<?, ?it/s]

Simulating steps:   0%|          | 0/61 [00:00<?, ?it/s]

Simulating steps:   0%|          | 0/19021 [00:00<?, ?it/s]

Simulating steps:   0%|          | 0/51 [00:00<?, ?it/s]

Simulating steps:   0%|          | 0/72001 [00:00<?, ?it/s]

In [ ]:
# plot the final state.
sim.plot_state(
    view_param=dict(elev=90, azim=0, roll=0),
    center_body=earth,
    center_view=True,
    zoom_factor=1.0,
    zoom_center=None,
    aspect_scale=(1, 1, 1),
    to_scale=False,
)

In [ ]:
# save as animation
sim.animate(
    steps_per_frame=300,
    framesep=50,
    view_param=dict(elev=90, azim=0, roll=0),
    center_body=earth,
    center_view=True,
    zoom_factor=1.0,
    zoom_center=None,
    aspect_scale=(1, 1, 1),
    unit=
    draw_acceleration=True,
    save_file="hohmann_transfer.mp4",
    plot_velocity=spacecrafts,
    legend="fig",
)